In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn
import math

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

/Users/yinchiahuang/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Take your stock, take 2017- 52 weeks
HMC = pd.read_csv('/Users/yinchiahuang/Library/Mobile Documents/com~apple~CloudDocs/CS677/Assignments/yinchia@bu.edu_hw1/HMC_Honda Motor Co., Ltd.csv')

In [3]:
HMC_2017 = HMC.copy()
HMC_2017['Date'] = pd.to_datetime(HMC_2017['Date'])
start_date = '12-28-2016'
end_date = '01-29-2018'
mask = (HMC_2017['Date'] > start_date) & (HMC_2017['Date'] <= end_date)
HMC_2017 = HMC_2017.loc[mask]
HMC_2017['Return'] = HMC_2017['Adj Close'].pct_change()
HMC_2017['Day of week'] = HMC_2017['Date'].dt.weekday_name
HMC_2017

,Date,Open,High,Low,Close,Adj Close,Volume,Return,Day of week
829,2016-12-29,29.420000,29.459999,29.230000,29.280001,28.236036,364900,NaN,Thursday
830,2016-12-30,29.200001,29.299999,29.090000,29.190001,28.149246,421200,-0.003074,Friday
831,2017-01-03,29.480000,29.610001,29.420000,29.610001,28.554270,864500,0.014388,Tuesday
832,2017-01-04,30.209999,30.670000,30.209999,30.660000,29.566833,705500,0.035461,Wednesday
833,2017-01-05,30.620001,30.780001,30.580000,30.660000,29.566833,482600,0.000000,Thursday
834,2017-01-06,30.350000,30.580000,30.240000,30.469999,29.383608,493600,-0.006197,Friday
835,2017-01-09,30.370001,30.500000,30.299999,30.430000,29.345034,585200,-0.001313,Monday
836,2017-01-10,30.139999,30.270000,30.020000,30.100000,29.026800,849600,-0.010845,Tuesday
837,2017-01-11,30.070000,30.180000,29.900000,30.180000,29.103947,809300,0.002658,Wednesday
838,2017-01-12,30.180000,30.290001,29.809999,30.160000,29.084658,737600,-0.000663,Thursday


In [4]:
date_index = HMC_2017["Date"].tolist()
date_index2 = pd.date_range('12-28-2016', periods=399, freq='D')

In [5]:
# find out the missing data 
HMC_2017_returns = pd.DataFrame({"Daily return": HMC_2017["Return"].tolist(), "Day of week":HMC_2017["Day of week"].tolist()}, index = date_index)
HMC_2017_returns = HMC_2017_returns.reindex(date_index2)
#HMC_2017_returns

In [6]:
# get vectors of weekly returns
number_of_week = [] 
weekly_returns = [] 

HMC_2017_returns = HMC_2017_returns.reset_index()

for row in range(5,len(HMC_2017_returns)):
    daily_return = HMC_2017_returns.loc[row]['Daily return'] 
    
    # replace missing value with previous data
    r = row -1
    while np.isnan(daily_return)== True:
        daily_return = HMC_2017_returns.loc[r]['Daily return']
        r -=1
        
    if row%7 == 5: # Monday --> create a new vector & add daily return in it 
        weekly_return = []
        weekly_return.append(daily_return)
    elif row%7 == 6 or row%7 == 0 or row%7 == 1: # Tues, Wed, Thurs --> add daily return in the vector

        weekly_return.append(daily_return)
    elif row%7 == 2 :# Friday --> add daily return in the vector & add the vector to a list of all vectors  
        weekly_return.append(daily_return)
        weekly_return = [round(elem, 4) for elem in weekly_return]
        weekly_returns.append(weekly_return)


In [7]:
# assign labels for each week --> I define a week is good(1) if monday return > 0, otherwise, it is a bad(0) week
week_labels = []

for i in range(len(weekly_returns)):
    monday_return = weekly_returns[i][0]

    if monday_return>0:
        week_labels.append(1)
    else:
        week_labels.append(0)

In [8]:
monday_returns = []
tuesday_returns = []
wednesday_returns = []
thursday_returns = []
friday_returns = [] 

for week in range(len(weekly_returns)):
    monday_returns.append(weekly_returns[week][0])
    tuesday_returns.append(weekly_returns[week][1])
    wednesday_returns.append(weekly_returns[week][2])
    thursday_returns.append(weekly_returns[week][3])
    friday_returns.append(weekly_returns[week][4])

In [9]:
#summarize all information
#month_year
data = {'week': list(range(1,57)), 'monday returns':monday_returns,'tuesday returns':tuesday_returns,'wednesday returns':wednesday_returns,'thursday returns':thursday_returns,'friday returns':friday_returns, 'daily returns':weekly_returns,'week label': week_labels}
df = pd.DataFrame(data=data)
df

,week,monday returns,tuesday returns,wednesday returns,thursday returns,friday returns,daily returns,week label
0,1,-0.0031,0.0144,0.0355,0.0000,-0.0062,"[-0.0031, 0.0144, 0.0355, 0.0, -0.0062]",0
1,2,-0.0013,-0.0108,0.0027,-0.0007,-0.0027,"[-0.0013, -0.0108, 0.0027, -0.0007, -0.0027]",0
2,3,-0.0027,-0.0140,0.0111,0.0120,0.0030,"[-0.0027, -0.014, 0.0111, 0.012, 0.003]",0
3,4,-0.0026,-0.0016,0.0158,-0.0019,-0.0212,"[-0.0026, -0.0016, 0.0158, -0.0019, -0.0212]",0
4,5,-0.0043,-0.0077,0.0081,0.0087,0.0278,"[-0.0043, -0.0077, 0.0081, 0.0087, 0.0278]",0
5,6,-0.0061,0.0165,0.0112,-0.0126,0.0077,"[-0.0061, 0.0165, 0.0112, -0.0126, 0.0077]",0
6,7,0.0177,-0.0037,-0.0069,-0.0091,0.0035,"[0.0177, -0.0037, -0.0069, -0.0091, 0.0035]",1
7,8,0.0035,0.0054,0.0025,-0.0075,-0.0054,"[0.0035, 0.0054, 0.0025, -0.0075, -0.0054]",1
8,9,-0.0067,-0.0086,0.0103,-0.0096,0.0052,"[-0.0067, -0.0086, 0.0103, -0.0096, 0.0052]",0
9,10,-0.0061,0.0036,-0.0042,0.0000,0.0036,"[-0.0061, 0.0036, -0.0042, 0.0, 0.0036]",0


In [10]:
df_2017 = df[:52]
df_2017_good = df_2017.loc[df['week label'] == 1]
df_2017_bad = df_2017.loc[df['week label'] == 0]

df_2018 = df[52:57]
df_2018 = df_2018.reset_index()

#### Naïve Bayesian Classification: Continuous Case

Bayes Formula:  P(c | x) = P(x | c) * P(c) / P(x)

Posterior = likelihood * Prior / Evidence

#### Classify new weeks as "good" week or "bad" week
- Posterior "Good" probability:
  P(G|X~) =  [P(G)P(Mon|G)P(Tue|G)P(Wed|G)P(Thur|G)P(Fri|G)] / evidence

- Posterior “Bad” probability:
  P(B|X~) =  [P(B)P(Mon|B)P(Tue|B)P(Wed|B)P(Thur|B)P(Fri|B)] / evidence

- The evidence is the normalization constant
  evidence = P(G|X~)  +  P(B|X~)

  This can be ignored (it is the same for both classes)

#### Method 1: self-written calculation

In [11]:
print('# of good weeks:' ,len(df_2017_good))
print('P(good)=',round(len(df_2017_good)/52,2))
p_good = len(df_2017_good)/52
print('\n# of bad weeks:' ,len(df_2017_bad))
p_bad = len(df_2017_bad)/52
print('P(bad)=',round(len(df_2017_bad)/52,2))

# of good weeks: 26
P(good)= 0.5

# of bad weeks: 26
P(bad)= 0.5


In [12]:
### column standard deviation  of the dataframe
print("class = good, standard deviation for each attribute\n")
print(df_2017_good.std(axis=0))

std_monday_good = df_2017_good['monday returns'].std()
std_tuesday_good = df_2017_good['tuesday returns'].std()
std_wednesday_good = df_2017_good['wednesday returns'].std()
std_thursday_good = df_2017_good['thursday returns'].std()
std_friday_good = df_2017_good['friday returns'].std()


print("\n\nclass = bad, standard deviation for each attribute\n")
print(df_2017_bad.std(axis=0))

std_monday_bad = df_2017_bad['monday returns'].std()
std_tuesday_bad = df_2017_bad['tuesday returns'].std()
std_wednesday_bad = df_2017_bad['wednesday returns'].std()
std_thursday_bad = df_2017_bad['thursday returns'].std()
std_friday_bad = df_2017_bad['friday returns'].std()

class = good, standard deviation for each attribute

week                 13.715628
monday returns        0.005368
tuesday returns       0.007570
wednesday returns     0.007965
thursday returns      0.009833
friday returns        0.005147
week label            0.000000
dtype: float64


class = bad, standard deviation for each attribute

week                 13.837574
monday returns        0.003435
tuesday returns       0.009823
wednesday returns     0.010907
thursday returns      0.007850
friday returns        0.009150
week label            0.000000
dtype: float64


In [13]:
### column mean  of the dataframe
print("class = good, mean for each attribute\n")
print(df_2017_good.mean(axis=0))

mean_monday_good = df_2017_good['monday returns'].mean()
mean_tuesday_good = df_2017_good['tuesday returns'].mean()
mean_wednesday_good = df_2017_good['wednesday returns'].mean()
mean_thursday_good = df_2017_good['thursday returns'].mean()
mean_friday_good = df_2017_good['friday returns'].mean()

print("\n\nclass = bad, mean for each attribute\n")
print(df_2017_bad.mean(axis=0))

mean_monday_bad = df_2017_bad['monday returns'].mean()
mean_tuesday_bad = df_2017_bad['tuesday returns'].mean()
mean_wednesday_bad = df_2017_bad['wednesday returns'].mean()
mean_thursday_bad = df_2017_bad['thursday returns'].mean()
mean_friday_bad = df_2017_bad['friday returns'].mean()

class = good, mean for each attribute

week                 33.038462
monday returns        0.006373
tuesday returns       0.002550
wednesday returns     0.001342
thursday returns     -0.001231
friday returns       -0.000562
week label            1.000000
dtype: float64


class = bad, mean for each attribute

week                 19.961538
monday returns       -0.003765
tuesday returns      -0.001731
wednesday returns     0.002573
thursday returns      0.000500
friday returns        0.001442
week label            0.000000
dtype: float64


In [14]:
def normdf(x,mean,sd):
    var = float(sd)**2
    pi = 3.1415926
    denom = (2*pi*var)**0.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom

In [15]:
def NBClassifier(daily_returns):
    p_mon_good = normdf(daily_returns[0],mean_monday_good,std_monday_good)
    p_tue_good = normdf(daily_returns[1],mean_tuesday_good,std_tuesday_good)
    p_wed_good = normdf(daily_returns[2],mean_wednesday_good,std_wednesday_good)
    p_thu_good = normdf(daily_returns[3],mean_thursday_good,std_thursday_good)
    p_fri_good = normdf(daily_returns[4],mean_friday_good,std_friday_good)
        
    p_mon_bad = normdf(daily_returns[0],mean_monday_bad,std_monday_bad)
    p_tue_bad = normdf(daily_returns[1],mean_tuesday_bad,std_tuesday_bad)
    p_wed_bad = normdf(daily_returns[2],mean_wednesday_bad,std_wednesday_bad)
    p_thu_bad = normdf(daily_returns[3],mean_thursday_bad,std_thursday_bad)
    p_fri_bad = normdf(daily_returns[4],mean_friday_bad,std_friday_bad)
    
    p_good_x = p_good * p_mon_good* p_tue_good * p_wed_good* p_thu_good*p_fri_good
    p_bad_x = p_bad * p_mon_bad* p_tue_bad * p_wed_bad* p_thu_bad * p_fri_bad
    
    #print("p_good_x = ",round(p_good_x))
    #print("p_bad_x =  ", round(p_bad_x))
    
    if p_good_x > p_bad_x:
        classification = 1
    else:
        classification = 0
    
    return classification

#### Test January, 2018 first 4 weeks 

In [16]:
predicted_week_labels = [] 
correct_points = 0
for row in range(len(df_2018)):
    predicted_week_label = NBClassifier(df_2018.loc[row]['daily returns'])
    if predicted_week_label == df_2018.loc[row]['week label']:
        correct_points +=1
    
    predicted_week_labels.append(predicted_week_label)
    

In [17]:
print("actual labels:",df_2018['week label'].tolist() )
print("predicted labels:",predicted_week_labels)

total_points = len(df_2018)
accuracy = 100.0*correct_points/total_points

print("Number of correct points out of a total {} points : {}, performance {:05.2f}%"
      .format(total_points, correct_points, accuracy))


actual labels: [0, 1, 0, 0]
predicted labels: [0, 1, 0, 0]
Number of correct points out of a total 4 points : 4, performance 100.00%


#### Test [0,0,0,0,0]

In [18]:
NBClassifier([0,0,0,0,0])

1

#### Method 2: Use GaussianNB()

In [19]:
X_train = df_2017
X_test = df_2018

gnb = GaussianNB()

used_features =['monday returns', 'tuesday returns', 'wednesday returns','thursday returns','friday returns'] 

gnb.fit(X_train[used_features].values, X_train["week label"])
y_pred = gnb.predict(X_test[used_features])

total = X_test.shape[0]
#mislabeled_points = X_test["Survived"] != y_pred).sum()

#### Test January, 2018 first 4 weeks 

In [20]:
X_test.loc[:, 'predicted'] = y_pred

comparison = (X_test['week label'] == X_test['predicted'])
X_test.loc[ :, 'correct_bool'] = comparison.values

X_test.loc[: , "correct"]=np.where(X_test.loc[:, "correct_bool"]==False,0,1)
correct_points = X_test['correct'].sum()
total_points = len(X_test)
accuracy = 100.0*correct_points/total_points

print("Number of correct points out of a total {} points : {}, performance {:05.2f}%"
      .format(total_points, correct_points, accuracy))

Number of correct points out of a total 4 points : 4, performance 100.00%


#### Test [0,0,0,0,0]

In [21]:
gnb.predict([[0,0,0,0,0]])

array([1])